In [1]:
from tensorflow.keras.models import load_model
model = load_model('pred.h5')

In [2]:
import pandas as pd

df = pd.read_csv("medical_data.csv")  # Make sure the path is correct


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

texts = df['Patient_Problem'].astype(str).tolist()  # Replace 'Problems' if column name is different

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

# Set max_length based on what you used during training, or estimate it
max_length = max([len(x.split()) for x in texts])


In [4]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [5]:
import pickle
from sklearn.preprocessing import LabelEncoder

# Assuming you have these columns in your DataFrame
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

df['Disease'] = label_encoder_disease.fit_transform(df['Disease'])
df['Prescription'] = label_encoder_prescription.fit_transform(df['Prescription'])

# Save them to files
with open("label_encoder_disease.pkl", "wb") as f:
    pickle.dump(label_encoder_disease, f)

with open("label_encoder_prescription.pkl", "wb") as f:
    pickle.dump(label_encoder_prescription, f)


In [6]:
with open("label_encoder_disease.pkl", "rb") as f:
    label_encoder_disease = pickle.load(f)

with open("label_encoder_prescription.pkl", "rb") as f:
    label_encoder_prescription = pickle.load(f)


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def make_prediction(patient_problem):
    try:
        sequence = tokenizer.texts_to_sequences([patient_problem])
        padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

        prediction = model.predict(padded_sequence)

        disease_index = np.argmax(prediction[0], axis=1)[0]
        prescription_index = np.argmax(prediction[1], axis=1)[0]

        disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
        prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

        return f"🩺 Predicted Disease: {disease_predicted}\n💊 Suggested Prescription: {prescription_predicted}"
    
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [20]:
import gradio as gr
import numpy as np

interface = gr.Interface(
    fn=make_prediction,
    inputs=gr.Textbox(lines=5, label="Describe Patient's Symptoms"),
    outputs="text",
    title="🧠 Medical Disease Predictor",
    description="Input symptoms and get a disease prediction with suggested prescription.",
)

interface.launch(share=True)  # Use inline=True inside Jupyter Notebook


* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://761db6159a7287f45d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
